In [20]:
import pandas as pd

from ib_insync import *
util.startLoop()
ib = IB()

In [54]:
%%time

cols = ['nsesymbol', 'lot', 'margin'] # standardize columns
exchange = 'NSE'

#...get the symbols
try:
    df_nse = pd.read_html('https://www.5paisa.com/5pit/spma.asp')[1].loc[:, [0,1,10]][1:].reset_index(drop=True)
except Exception:
    df_nse = pd.read_html('https://www.tradeplusonline.com/Equity-Futures-Margin-Calculator.aspx')[1][2:]
    df_nse = df_nse[df_nse.columns[[0,1,6]]]

df_nse.columns=cols # standardized columns

df_nse = df_nse.assign(lot=df_nse.lot.astype('float64'), margin=df_nse.margin.astype('float64'))

#...qualify the underlyings
# make the symbols
df_nse = df_nse.assign(symbol=df_nse.nsesymbol.str.slice(0,9))

# nse symbol to ibkr symbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap nse symbol to ibkr symbol
df_nse.symbol = df_nse.symbol.replace(ntoi)

# separate indexes and equities, eliminate discards from df_slm
indexes = ['NIFTY50', 'BANKNIFTY']
discards = ['NIFTYMID5', 'NIFTYIT', 'LUPIN']
equities = sorted([s for s in df_nse.symbol if s not in indexes+discards])

symbols = equities+indexes

cs = [Stock(s, exchange) if s in equities else Index(s, exchange) for s in symbols]

with ib.connect('127.0.0.1', 3000, 10):
    und_contracts = ib.qualifyContracts(*[c for c in cs])
    tickers = ib.reqTickers(*und_contracts)
    hist = [ib.reqHistoricalData(contract=c, endDateTime='', durationStr='365 D',
                                barSizeSetting='1 day', whatToShow='Trades', useRTH=True) for c in und_contracts][0]

Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests


Wall time: 7min 59s


In [53]:
tickers

[Ticker(contract=Stock(conId=44652144, symbol='ACC', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ACC', tradingClass='ACC'), time=datetime.datetime(2019, 3, 27, 13, 22, 56, 403097, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=1602.55, lastSize=150, open=1584.0, close=1570.85, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[]),
 Ticker(contract=Stock(conId=56986798, symbol='ADANIENT', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ADANIENT', tradingClass='ADANIENT'), time=datetime.datetime(2019, 3, 27, 13, 23, 1, 792801, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=137.5, lastSize=100, open=138.55, close=138.3, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[]),
 Ticker(contract=Stock(conId=79403918, symbol='ADANIPORT', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ADANIPORTS', tradingClass='ADANIPORT'), time=datetime.datetime(2019, 

In [ ]:
%%time
# Program to get connected to appropriate IB instance
import pandas as pd
import math
from itertools import product, repeat

from ib_insync import *

exchange = 'NSE'
tradingdays = 252

#... build the symbols
#______________________

tp = pd.read_html('https://www.tradeplusonline.com/Equity-Futures-Margin-Calculator.aspx')

df_tp = tp[1][2:].iloc[:, :-1]
df_tp = df_tp.iloc[:, [0,1,5]]
df_tp.columns=['nseSymbol', 'lot', 'margin']

cols = df_tp.columns.drop('nseSymbol')
df_tp[cols] = df_tp[cols].apply(pd.to_numeric, errors='coerce') # convert lot and margin to numeric

df_slm = df_tp.copy()

# Truncate to 9 characters for ibSymbol
df_slm['ibSymbol'] = df_slm.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_slm.ibSymbol = df_slm.ibSymbol.replace(ntoi)

# separate indexes and equities, eliminate discards from df_slm
indexes = ['NIFTY50', 'BANKNIFTY']
discards = ['NIFTYMID5', 'NIFTYIT', 'LUPIN']
equities = sorted([s for s in df_slm.ibSymbol if s not in indexes+discards])

symbols = equities+indexes

cs = [Stock(s, exchange) if s in equities else Index(s, exchange) for s in symbols]

util.startLoop()
with IB() as ib:
    
    ib.connect('127.0.0.1', 3000, clientId=13) # For nse - Kavi

    qcs = ib.qualifyContracts(*cs) # qualified underlyings

    symbol = 'PNB'
    qualund = [q for q in qcs if q.symbol == symbol][0]

    ib.reqMktData(qualund, '456', False, False)

    ticker = ib.ticker(qualund)

    while math.isnan(ticker.close):
        while ticker.dividends is None:
            ib.sleep(0.2)


    hist = ib.reqHistoricalData(contract=qualund, endDateTime='', 
                                    durationStr='365 D', barSizeSetting='1 day',  
                                                whatToShow='Trades', useRTH=True)

    df_ohlc = util.df(hist).set_index('date').sort_index(ascending=False)

    # get cumulative standard deviation
    df_stdev = pd.DataFrame(df_ohlc['close'].expanding(1).std(ddof=0))
    df_stdev.columns = ['stdev']

    # get cumulative volatility
    df_vol = pd.DataFrame(df_ohlc['close'].pct_change().expanding(1).std(ddof=0)*math.sqrt(tradingdays))
    df_vol.columns = ['volatility']
    df_ohlc = df_ohlc.join(df_vol)
    df_ohlc = df_ohlc.join(df_stdev)


    chains = ib.reqSecDefOptParams(underlyingSymbol=qualund.symbol, futFopExchange='', 
                                  underlyingConId=qualund.conId, underlyingSecType=qualund.secType)


    chain = [c for c in chains if c.exchange == exchange]

    right = ['C', 'P']

    sekre = [list(product([symbol], c.expirations, c.strikes, r, [exchange])) for s, c, r, X in zip(symbol, chain, right, exchange)][0]

    contracts = ib.qualifyContracts([Option(s, e, k, r, e) for s, e, k, r, e in sekre])

In [ ]:
ib.disconnect()

In [ ]:
%%time
import math
from ib_insync import *
util.startLoop()

with IB() as ib:
    
    ib.connect('127.0.0.1', 3000, clientId=13) # For nse - Kavi
    und = Stock(symbol='PNB', exchange='NSE' )
    qualund = ib.qualifyContracts(und)[0]
    
    ib.reqMktData(qualund, '456', False, False)
    ticker = ib.ticker(qualund)

    hist = ib.reqHistoricalData(contract=qualund, endDateTime='', 
                                    durationStr='365 D', barSizeSetting='1 day',  
                                                whatToShow='Trades', useRTH=True)
    chains = ib.reqSecDefOptParams(underlyingSymbol=qualund.symbol, futFopExchange='', 
                                  underlyingConId=qualund.conId, underlyingSecType=qualund.secType)
    
    while math.isnan(ticker.close):
        while ticker.dividends is None:
            ib.sleep(0.2)
            
    print(qualund, ticker, list(hist), chains)

In [ ]:
ib = IB()
help(ib.ticker)